# 서울시 생활인구
## 서울 생활인구 현황 (2022.09.28. 기준)
### url: https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do

1. 집계구 단위 서울 생활인구(내국인)
    - url: https://data.seoul.go.kr/dataList/OA-14979/F/1/datasetView.do
    - 설명: 서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 내국인

<!-- <br> -->


2. 집계구 단위 서울 생활인구(장기체류 외국인)
    - url: https://data.seoul.go.kr/dataList/OA-14978/F/1/datasetView.do
    - 설명:서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 장기체류 외국인

<!-- <br> -->


3. 집계구 단위 서울 생활인구(단기체류 외국인)
    - url: https://data.seoul.go.kr/dataList/OA-14980/F/1/datasetView.do
    - 설명:서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 단기체류 외국인

<!-- <br> -->



※ 개인정보 비 식별화를 위하여 ‘3명’ 이하인 경우 “ * ” 처리


In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
from tqdm.auto import tqdm, trange
from time import sleep

## 1. 집계구 단위 서울 생활인구(내국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년
    - 2019년 10월 15-27일 데이터 없음
## 2. 집계구 단위 서울 생활인구(장기체류 외국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년
## 3. 집계구 단위 서울 생활인구(단기체류 외국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년

In [3]:
# 폴더 안 파일명 리스트 가져오기
local_people_2019_list = []
local_people_2020_list = []
local_people_2021_list = []

long_foreigner_list = []
temp_foreinger_list= []
pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/local_people/2019"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2019_list.append(file)
        
pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/local_people/2020"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2020_list.append(file)
        
pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/local_people/2021"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2021_list.append(file)

pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/long_foreigner/"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        long_foreigner_list.append(file)
 
pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/temp_foreigner/"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        temp_foreinger_list.append(file)

local_people_2019_list
sort(local_people_2020_list)
sort(local_people_2021_list)
sort(long_foreigner_list)
sort(temp_foreinger_list)

print("local_people_2019_list >>", len(local_people_2019_list))
print("local_people_2020_list >>", len(local_people_2020_list))
print("local_people_2021_list >>", len(local_people_2021_list))

print("long_foreigner_list >>", long_foreigner_list)
print("temp_foreinger_list >>", temp_foreinger_list)

  0%|          | 0/352 [00:00<?, ?it/s]

  0%|          | 0/366 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

NameError: name 'sort' is not defined

In [ ]:
def data_process(file_list, path):
    df_mean_per_day_list = []
    problem_list = []
    pbar = tqdm(file_list)
    for file in pbar:
        pbar.set_description(file) # pbar description에 작업중인 파일명 업데이트
        try:df_temp = pd.read_csv(path + file, encoding="utf-8", na_values="*")
        except: 
            # df_temp = pd.read_csv(path + file, encoding="euc-kr", na_values="*")
            print("encoding문제 파일 >>", file)
            problem_list.append(file)
            continue
        
        #결측값 0으로 대체
        df_temp.fillna(0, inplace=True)         
        df_temp = create_dates(df_temp)
        df_mean_per_day = combine_ages(df_temp)
        df_mean_per_day_list.append(df_mean_per_day)
    df_concat = pd.concat(df_mean_list)
    return df_concat

In [ ]:
# 기준일ID 컬럼을 date형식으로 변환
# 년, 월 컬럼 생성
def create_dates(df_temp):
    df_temp.insert(0, '날짜', pd.to_datetime(df_temp['기준일ID'], format='%Y%m%d'))
    df_temp.insert(1, '년', df_temp['날짜'].dt.year)
    df_temp.insert(2, '월', df_temp['날짜'].dt.month)

    # 주말 여부 컬럼 생성
    df_temp.insert(3, "주말", df_temp['날짜'].dt.dayofweek > 4)

    # 기준일ID컬럼 drop
    df_temp.drop(columns=['기준일ID', '시간대구분'], inplace=True)
    return df_temp

In [ ]:
def combine_ages(df_temp):
    cols = [
    '남자0세부터9세생활인구수', '여자0세부터9세생활인구수', 
    '남자10세부터14세생활인구수', '남자15세부터19세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수', 
    '남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '여자20세부터24세생활인구수', '여자25세부터29세생활인구수',
    '남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수', 
    '남자40세부터44세생활인구수', '남자45세부터49세생활인구수', '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', 
    '남자50세부터54세생활인구수', '남자55세부터59세생활인구수', '여자50세부터54세생활인구수', '여자55세부터59세생활인구수', 
    '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', 
    '남자70세이상생활인구수', '여자70세이상생활인구수'
    ]
    df_temp['10세 이하'] = df_temp.남자0세부터9세생활인구수 + df_temp.여자0세부터9세생활인구수
    df_temp['10대'] = df_temp.남자10세부터14세생활인구수 + df_temp.남자15세부터19세생활인구수 + df_temp.여자10세부터14세생활인구수 + df_temp.여자15세부터19세생활인구수
    df_temp['20대'] = df_temp.남자20세부터24세생활인구수 + df_temp.남자25세부터29세생활인구수 + df_temp.여자20세부터24세생활인구수 + df_temp.여자25세부터29세생활인구수
    df_temp['30대'] = df_temp.남자30세부터34세생활인구수 + df_temp.남자35세부터39세생활인구수 + df_temp.여자30세부터34세생활인구수 + df_temp.여자35세부터39세생활인구수
    df_temp['40대'] = df_temp.남자40세부터44세생활인구수 + df_temp.남자45세부터49세생활인구수 + df_temp.여자40세부터44세생활인구수 + df_temp.여자45세부터49세생활인구수
    df_temp['50대'] = df_temp.남자50세부터54세생활인구수 + df_temp.남자55세부터59세생활인구수 + df_temp.여자50세부터54세생활인구수 + df_temp.여자55세부터59세생활인구수
    df_temp['60대'] = df_temp.남자60세부터64세생활인구수 + df_temp.남자65세부터69세생활인구수 + df_temp.여자60세부터64세생활인구수 + df_temp.여자65세부터69세생활인구수
    df_temp['70세 이상'] = df_temp.남자70세이상생활인구수 + df_temp.여자70세이상생활인구수
    df_temp.drop(columns=cols, inplace=True)
    df_mean_per_day = df_temp.groupby(['날짜', '년', '월', '주말', '행정동코드', '집계구코드']).mean()
    return df_mean_per_day

In [ ]:
# #define how to aggregate various fields
# agg_functions = {'년':'first', '월':'first', '주말':'first', '행정동코드': 'first', '집계구코드':'first', '10세 이하': 'sum', '10대': 'sum', '20대': 'sum', '30대': 'sum', '40대': 'sum', '50대': 'sum', '60대': 'sum', '70세 이상': 'sum', '총생활인구수': 'sum'}

# #create new DataFrame by combining rows with same id values
# df_new = df_temp.groupby(df_temp['날짜']).aggregate(agg_functions)
# df_new

In [ ]:
local_people_2019_df = data_process(local_people_2019_list, "./data/집계동 단위 서울 생활인구/local_people/2019/")

In [ ]:
# local_people_2019_df = concat_files(local_people_2019_list, "./data/집계동 단위 서울 생활인구/local_people/2019/")
# local_people_2020_df = concat_files(local_people_2020_list, "./data/집계동 단위 서울 생활인구/local_people/2020/")
# local_people_2021_df = concat_files(local_people_2021_list, "./data/집계동 단위 서울 생활인구/local_people/2021/")


# long_foreigner_df = concat_files(long_foreigner_list, "./data/집계동 단위 서울 생활인구/long_foreigner/")
# temp_foreinger_df = concat_files(temp_foreinger_list, "./data/집계동 단위 서울 생활인구/temp_foreigner/")
# print("local_people_df.shape >>", local_people_df.shape)
# print("long_foreigner_df.shape >>", long_foreigner_df.shape)
# print("temp_foreinger_df.shape >>", temp_foreinger_df.shape)

In [ ]:
# # 결측값 0으로 대체
# local_people_df.fillna(0, inplace=True)
# local_people_df.info()

In [ ]:
# for col in tqdm(cols):
#     if col

In [ ]:
# local_people_df['10대 남성'] = local_people_df.남자10세부터14세생활인구수 + local_people_df.남자15세부터19세생활인구수
# local_people_df['20대 남성'] = 